In [2]:
!pip install -qU langchain langchain_openai langchain-community langgraph arxiv langchain-google-community googlesearch-python

## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [4]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [9]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("GOOGLE_API_KEY")

In [6]:
os.environ["GOOGLE_CSE_ID"] = getpass.getpass("GOOGLE_CSE_ID")

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE5 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

In [13]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import Tool

tavily_tool = TavilySearchResults(max_results=5)
google_search = Tool(
    name="GoogleSearch",
    func=GoogleSearchAPIWrapper().run, # Use the .run method directly
    description="Use this tool to search Google.", # Provide a description
)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
    google_search,
]

# tool2 = tool = Tool(
#     name="google_search",
#     description="Search Google for recent results.",
#     func=google_search.run,
# )
# tool2.run("Obama's first name?")

In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [15]:
model = model.bind_tools(tool_belt)

In [16]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

In [17]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

In [18]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

In [19]:
uncompiled_graph.set_entry_point("agent")

In [20]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

In [21]:
uncompiled_graph.add_edge("action", "agent")

All that's left to do now is to compile our workflow - and we're off!

In [22]:
compiled_graph = uncompiled_graph.compile()

In [ ]:
from langchain_core.messages import HumanMessage

#inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Tavily!")]}
inputs = {"messages" : [HumanMessage(content="A new study claims that drinking coffee can increase your risk of developing cancer. You will perform Claim extraction to get the claims and statements from the news article that needs to be verified using Tavily, then perform Evidence search using Arxiv for the QLoRA paper and publication that support or refute the claim and finally perform fact-checking by matching claims with reliable sources such as government or fact checking websites that corroborate or debunk those claims. Finally you will generate a report that includes claims made in the news article, evidence that was found to support or debunk the claim and finally as assessment of each claim based on research to be True, False, Partially True or Unverified ")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")